In [ ]:
import sys, os, glob, warnings, time, datetime
from math import sqrt
import numpy as np
import scipy, mne
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import io, stats, interpolate
%matplotlib inline
from sklearn.pipeline import make_pipeline
from sklearn.svm import LinearSVC, SVC
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import LeaveOneOut, cross_val_score, KFold, cross_val_predict
from sklearn.preprocessing import StandardScaler, scale
from scipy.signal import savgol_filter
warnings.filterwarnings('ignore')

In [ ]:
project_path = 'D:/Dropbox/Projects/featureReplay/'
SUBJECTS = ['S01','S02','S03','S04','S05','S06','S07','S08','S09','S10',
            'S11','S12','S13','S14','S15','S16','S17','S18','S19','S20',
            'S21','S22','S23','S24','S25','S26','S27','S28','S29','S30',
            'S31','S32','S33','S34','S35']

# !! Important: if you only select one subject, you must still write as [n-1:n]
selected_subj = SUBJECTS[:18]
n_subjects    = len(selected_subj)
print(['Running Subjects:'] + selected_subj)

# Get selected channel index (occipital)
chans_all       = np.loadtxt(project_path + 'data_v5/misc_data/channels_all.txt', dtype='str')
chans_occipital = np.loadtxt(project_path + 'data_v5/misc_data/occipital_channels.txt', dtype='str')
chans_without_frontal = np.loadtxt(project_path + 'data_v5/misc_data/channels_without_frontal.txt', dtype='str')
chans_idx       = np.where(np.in1d(chans_all, chans_occipital) == True)[0]

optimal_time_matrix = np.load(project_path + 'data_v5/saved_source_data/optimal_time_idx_occipital_acc_matrix.npy')
actual_subj = ['S01','S02','S03','S04','S05','S06','S07','S08','S09','S10','S11','S12','S13','S14','S15','S16','S17','S18','S19','S20','S22','S23','S24','S25']
subj_idx = np.nonzero(np.in1d(selected_subj, selected_subj))[0]

### Loading data for plotting

In [ ]:
f_mat = scipy.io.loadmat(project_path + 'data_v5/saved_source_data/preds_modelTrain.mat')
proba = f_mat['preds']*100 # n_subjs, n_true_label, n_predict, n_times
proba_mean = np.mean(proba, axis=0)
proba_sem  = stats.sem(proba, axis=0)

f_mat = scipy.io.loadmat(project_path + 'data_v5/saved_source_data/preds_modelTrain_perm.mat')
proba_perm = f_mat['preds_perm']*100 # n_subjs, n_permutation_times, n_true_label, n_predict, n_times
percentile = np.percentile(proba_perm, 95, axis=1) # supra-threshold 95% across permutation_times
percentile2 = np.percentile(percentile, 95, axis=3) # supra-threshold 95% across time points
percentile_mean = np.mean(percentile2, axis=(0,2))

times = np.tile(np.arange(proba.shape[3]),(4,1)).T

### Plot mean decoding probability

In [ ]:
# shift all decoding probabilities at the optimal time point to an arbitrary 200 ms (index = 125)
proba_shifted = np.zeros(proba.shape)
for isubj in range(n_subjects):
    for ilabel in range(4): # true label
        proba_shifted[isubj,ilabel,:,:] = np.roll(proba[isubj,ilabel,:,:], 125-int(optimal_time_matrix[subj_idx[isubj],ilabel]), axis=1)

# calculate mean, SEM etc.
proba_mean_shifted = np.mean(proba_shifted, axis=0)
proba_sem_shifted = stats.sem(proba_shifted, axis=0)

In [ ]:
# start plotting here
fig, axs = plt.subplots(nrows=1,ncols=4,sharex=True,sharey=True,figsize=(4*6, 1*6))
plt.rcParams["font.family"] = "arial"

for i in range(4): # true label
    ax = axs[i]
    
    for j in range(4): # predicted label probability
        if i == 0: # plot legend in fig.1 only
            ax.plot(proba_mean_shifted[i,j], lw=1.5, label='Model %s$^{\circ}$' %(j*90))
            ax.legend(loc=1,fontsize=14,frameon=False)
        else:
            ax.plot(proba_mean_shifted[i,j], lw=1.5)
        ax.fill_between(times[:,j], proba_mean_shifted[i,j]-proba_sem_shifted[i,j], proba_mean_shifted[i,j]+proba_sem_shifted[i,j], edgecolor='none',alpha=.35)

    ax.text(1, 23, 'true label = %s$^{\circ}$' %(i*90), fontsize=14)
    ax.axhline(percentile_mean[i], color='k', linestyle='--', alpha=0.5, lw=1.5)

    ax.set_xticks(np.arange(25,proba.shape[3]+1, step=50)) # convert volumes to seconds
    ax.set_xticklabels(('-0.2', '0', '0.2', '0.4', '0.6', '0.8', '1.0'))
    ax.set_ylim([7, 25])
    ax.set_xlabel('Time relative to stimulus onset (s)', fontsize=14);
    ax.set_ylabel('Predicted probability (%)', fontsize=14);
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis = 'both', which = 'major', direction='in', top=False, right=False, labelsize = 14)

## save figures
save_figure = False
if save_figure:
    fig_path = project_path + '/data_v5/saved_figures/modelTrain_rs250_ica_occipital/'
    if not os.path.exists(fig_path): os.makedirs(fig_path);
    fig.savefig(fig_path+"modelTrain_proba_mean_shifted_matrixOfAcc_raw.pdf", bbox_inches='tight',dpi=300)

### Plot the smoothed data for visualization

In [ ]:
# start plotting here
fig, axs = plt.subplots(nrows=1,ncols=4,sharex=True,sharey=True,figsize=(4*6, 1*6))
plt.rcParams["font.family"] = "arial"

for i in range(4): # true label
    ax = axs[i]
    
    for j in range(4): # predicted label probability
        y_filtered = savgol_filter(proba_mean_shifted[i,j], window_length=7, polyorder=1) # Apply Savitzky-Golay filter 
        if i == 0: # plot legend in fig.1 only
            ax.plot(y_filtered, lw=1.5, label='Model %s$^{\circ}$' %(j*90))
            ax.legend(loc=1,fontsize=14,frameon=False)
        else:
            ax.plot(y_filtered, lw=1.5)
        ax.fill_between(times[:,j], y_filtered-proba_sem_shifted[i,j], y_filtered+proba_sem_shifted[i,j], edgecolor='none',alpha=.35)

    ax.text(1, 23, 'true label = %s$^{\circ}$' %(i*90), fontsize=14)
    ax.axhline(percentile_mean[i], color='k', linestyle='--', alpha=0.5, lw=1.5)

    ax.set_xticks(np.arange(25,proba.shape[3]+1, step=50)) # convert volumes to seconds
    ax.set_xticklabels(('-0.2', '0', '0.2', '0.4', '0.6', '0.8', '1.0'))
    ax.set_ylim([7, 25])
    ax.set_xlabel('Time relative to stimulus onset (s)', fontsize=14);
    ax.set_ylabel('Predicted probability (%)', fontsize=14);
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(axis = 'both', which = 'major', direction='in', top=False, right=False, labelsize = 14)

## save figures
save_figure = False
if save_figure:
    fig_path = project_path + '/data_v5/saved_figures/modelTrain_rs250_ica_occipital/'
    if not os.path.exists(fig_path): os.makedirs(fig_path);
    fig.savefig(fig_path+"modelTrain_proba_mean_shifted_matrixOfAcc_smoothed.pdf", bbox_inches='tight',dpi=300)